In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from Modules.dataEngineering import dataEngineering
dataEngineering = dataEngineering()

In [2]:
df_ratings        = dataEngineering.loadRatings()['data']

In [3]:
from surprise import Dataset, SVD, Reader
from surprise.model_selection import GridSearchCV

# Use movielens-100K
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['UserID', 'MovieID', 'userRating']], reader)

# param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
# gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)

# gs.fit(data)

# # best RMSE score
# print(gs.best_score["rmse"])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params["rmse"])

0.8902769849552981
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}

In [4]:
# gs.best_params

In [5]:
option_svd = {
    'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4
}

In [6]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [7]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

model_SVD = SVD(n_epochs=option_svd['n_epochs'], lr_all=option_svd['lr_all'], reg_all=option_svd['reg_all'],)

model_SVD.fit(train_set)

predictions = model_SVD.test(test_set)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.8916
MAE:  0.6888


In [8]:
def predict_model(pivot):
    matrix = pivot.copy()
    u_col = 0
    ratings = []
    for u_id, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            m_id = matrix.columns[m_col]
            if pd.isnull(item):
                prep_rating = round(model_SVD.predict(u_id, m_id).est, 4)
                ratings.append([u_id, m_id, prep_rating])
                matrix.iloc[u_col][m_id] = prep_rating
        u_col += 1
    
    recomment_unwatched = pd.DataFrame(ratings, columns=['UserID', 'MovieID', 'prep_Rating'])
    return matrix, recomment_unwatched.sort_values(by=['prep_Rating'], ascending=False)

In [9]:
df_pivot = df_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

In [10]:
mat, prep = predict_model(df_pivot.iloc[1:2])

In [11]:
mat

MovieID,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
UserID,,,,,,,,,,,,,,,,,,,,,
2,3.958,3.6884,3.5217,3.5588,3.4795,4.0096,3.3964,3.5006,3.4758,3.6509,...,3.7324,3.6817,3.7765,3.6873,3.6318,3.7821,3.6665,3.7408,3.7198,3.6817


In [12]:
prep[prep["UserID"] == 2].head(10)

,UserID,MovieID,prep_Rating
903,2,1204,4.2952
972,2,1276,4.2585
838,2,1104,4.2441
2221,2,2959,4.2337
600,2,750,4.2231
692,2,912,4.2218
411,2,475,4.2190
895,2,1196,4.2171
896,2,1197,4.2024
906,2,1208,4.2005


Save Model

In [13]:
import joblib

joblib.dump(model_SVD, 'models/model_SVD.pkl')

['models/model_SVD.pkl']